In [4]:
import glob
import os.path
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path

from PIL import Image
from tqdm.notebook import tqdm


class AppStorePreviewImageGenerator:
  SOURCE_IMAGES_ROOT = '../Screenshots for app store/sources'
  DESTINATION_IMAGES_ROOT = '../Screenshots for app store/generated'

  BUTTONS = {
      'gamepad': {
          'path': '../Bundle/JSDPad/dPad-None@2x.png',
          'offset': (20, -20-150),  # +x = left, -x = right, +y = top, -y = bottom
          'size': (150, 150),
      },
      'esc': {
          'path': '../Screenshots for app store/esc button@2x.png',
          'offset': (-20-50, 20),
          'size': (50, 50),
      },
      'return': {
          'path': '../Screenshots for app store/return button@2x.png',
          'offset': (-20-50, -20-50),
          'size': (50, 50),
      },
      'tab': {
          'path': '../Screenshots for app store/tab button@2x.png',
          'offset': (20, 20),
          'size': (50, 50),
      },
  }
  DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT = {
      '5.5': (401, 2, 2208, 1242),
      '6.5': (458, 3, 2688, 1242),
      '12.9': (264, 2, 2732, 2048),
  }
  OPACITY = round(.125 * 255)

  def generate_images(self):
    params = list(self._processing_params_generator())
    with ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(self._process_image, params), total=len(params)))
    return results

  def _processing_params_generator(self):
      for diagonal in self.DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT:
        source_image_paths = self._generate_source_image_paths(diagonal)
        output_directory = Path(os.path.join(self.DESTINATION_IMAGES_ROOT, diagonal))
        output_directory.mkdir(parents=True, exist_ok=True)

        for source_image_path in source_image_paths:
            yield diagonal, output_directory, source_image_path

  def _process_image(self, params):
      diagonal, output_directory, source_image_path = params
      image = self.generate_image(source_image_path, diagonal)
      destination_image_path = os.path.join(
          output_directory, os.path.basename(source_image_path))
      image.save(destination_image_path)

      return image

  def generate_image(self, source_image_path, diagonal):
      bg_image = Image.open(source_image_path)
      dpi, scale, *size = self.DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT[diagonal]

      for name, data in self.BUTTONS.items():
          self._process_button(bg_image, data, size, scale)
      bg_image = bg_image.convert('RGB')
      return bg_image

  def _generate_source_image_paths(self, diagonal):
    source_image_paths = glob.glob(os.path.join(self.SOURCE_IMAGES_ROOT, diagonal, '*.png'))
    return source_image_paths

  def _process_button(self, bg_image, image_data, size, scale):
    real_size = [d*scale for d in image_data['size']]
    image = Image.open(image_data['path']).resize(real_size).convert(mode='RGBA')
    actual_offset = [
      self._calculate_actual_offset(o, s, scale)
      for (o, s) in zip(image_data['offset'], size)
    ]

    image_data = list(image.getdata())
    for i, pixel in enumerate(image_data):
      image_data[i] = pixel[:3] + (self.OPACITY, ) if pixel[3] > 0 else pixel
    image.putdata(image_data)

    bg_image.paste(image, box=actual_offset, mask=image)

  def _calculate_actual_offset(self, synth_offset, bg_image_size, scale):
                                                                     """'+' because `synth_offset` is less than zero and will decrease value on sum."""
                                                                     scaled_offset = synth_offset * scale
                                                                     return scaled_offset if synth_offset > 0 else bg_image_size + scaled_offset

In [5]:
class SourceImageGenerator(AppStorePreviewImageGenerator):
  SOURCE_IMAGES_ROOT = '../Screenshots for app store/Кандидаты на скрины из notion'
  DESTINATION_IMAGES_ROOT = '../Screenshots for app store/sources'

  def generate_image(self, source_image_path, diagonal):
      image = Image.open(source_image_path)
      _, _, *size = self.DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT[diagonal]
      x_scale, y_scale = [size[i] / image.size[i] for i in range(len(size))]
      scale = x_scale if x_scale >= y_scale else y_scale
      image = image.resize([round(i*scale) for i in image.size])
      x_offset = abs(size[0]-image.size[0])
      image = image.crop((x_offset, 0, size[0]+x_offset, size[1]))

      return image

  def _generate_source_image_paths(self, diagonal):
    source_image_paths = glob.glob(os.path.join(self.SOURCE_IMAGES_ROOT, '*.png'))
    return source_image_paths

In [6]:
SourceImageGenerator().generate_images()
AppStorePreviewImageGenerator().generate_images()

[<PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D8745F8>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D8A7470>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D8A73C8>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D888CC0>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D874080>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D8A7400>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D8A70B8>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D8740F0>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D888B38>,
 <PIL.Image.Image image mode=RGB size=2208x1242 at 0x10D874F98>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x10D3A32B0>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x10D8740B8>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x10D874908>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x10D874F60>,
 <PIL.Image.Image image mode=RGB size=2688x1242 at 0x10D874BA8>,
 <PIL.Image.Image image m